In [18]:
import pandas as pd
import os
import math
import numpy as np
import networkx as nx
import my_nx as my_nx
import matplotlib.pyplot as plt
from datetime import datetime,time, timedelta
import pickle
from process_transfers import get_merged_stops, get_merged_stops_names
from heapq import heappop, heappush
import geopandas as gpd



In [19]:
weekday_subway_network = pickle.load(open('subway_network_weekday.pickle', 'rb'))
weekend_subway_network = pickle.load(open('subway_network_weekend.pickle', 'rb'))
weekday_bus_network = pickle.load(open('bus_network_weekday.pickle', 'rb'))
weekend_bus_network = pickle.load(open('bus_network_weekend.pickle', 'rb'))

In [21]:
def compute_shortest_path(G, source, target, transfer_time=6, stop_time=1):
    def dijkstra_with_transfers(G, source, target):
        
        # initilize the distance/travel time of every stop from the source stop to inifinity
        distances = {node: float('inf') for node in G.nodes()}
        distances[source] = 0
        
        
        pq = [(0, source, None)]  # (distance, current_node, current_edge_routes)
        
        # keep track of the previous stop of the shortest path
        previous_nodes = {node: None for node in G.nodes()}
        
        while pq:
            # print(pq)
            current_distance, current_node, current_edge_routes = heappop(pq)
            
            
            if current_distance > distances[current_node]:
                continue
            
            for neighbor in G.neighbors(current_node):
                weight = G[current_node][neighbor]['travel_time']
                edge_routes = set(G[current_node][neighbor]['routes'])
                
                # use to see if a transfer is needed
                if current_edge_routes and not current_edge_routes.intersection(edge_routes):
                    weight += transfer_time  # Add transfer time if no common routes
                
                new_distance = current_distance + weight + stop_time
                
                if new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    previous_nodes[neighbor] = current_node
                    heappush(pq, (new_distance, neighbor, edge_routes))
        
        path = []
        current_node = target
        while current_node is not None:
            path.append(current_node)
            current_node = previous_nodes[current_node]
        path.reverse()
        
        return path, distances[target]

    shortest_path, total_travel_time = dijkstra_with_transfers(G, source, target)
    
    return shortest_path, total_travel_time

In [25]:
path, time = compute_shortest_path(weekday_subway_network, '101','101')
path
# time

['101']

In [23]:
def plot_from_pickle(network, ax, path):
    pos=nx.get_node_attributes(network, 'pos')
    name = nx.get_node_attributes(network, 'name')
    edge_labels = {(u, v): f"{d['vechicle_trips']}, {d['travel_time']}" for u, v, d in network.edges(data=True)}
    
    temp_path = [(path[i], path[i+1]) for i in range(len(path)-1)]
    print(temp_path)
    edge_colors = ['red' if (u,v) in temp_path else 'grey' for u,v,data in network.edges(data=True)]
    
    nx.draw(network, pos, ax=ax, edge_color=edge_colors, node_size=9, node_color='blue', labels=name, font_color='purple', font_size='12', with_labels=False, arrowsize=4, verticalalignment='bottom', connectionstyle='arc3, rad = 0.1', label="Bus Station")
    
    # my_nx.my_draw_networkx_edge_labels(network, pos=pos, ax=ax, edge_labels=edge_labels, font_color='red', font_size=4, rotate=True, rad=0.1)
    
    # plt.title(f"{title}")
    # plt.show()
    
# fig, ax = plt.subplots(figsize=(15,10))
# plot_from_pickle(weekday_subway_network, ax, path)
